# Content Collection and Processing for TSE-NER Long-tail Entity Extraction

Here we will try to follow the full process required to train and use a NER model.

In [23]:
# This is in case you update modules while working
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Content Collection (a.k.a. crawling a *bunch* of papers)

For this step we will be using [sci-paper-miner](https://github.com/ronentk/sci-paper-miner) to download scientific publications from arXiv. Get the code from their repository, and use the following command to run it:

`python crawl_core.py <your-api>` 

In the `crawl_core` file, you can modify the topics and range of years that you want to download, in `configs` you can write a name for the folder where the data will be stored.

In [24]:
import json
import os

In [32]:
# This is the path where all the json files were downloaded 

path = 'sci_paper_miner/data/arxiv_2006-2017_cs/raw_query/'

We know that the files we downloaded are in JSON format, but let's check their structure.
For this we can use `os.walk`.

In [41]:
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.json'):
            print(filename)

144_Computer Science - Artificial Intelligence_2017_5_0.json
144_Computer Science - Artificial Intelligence_2017_5_1.json
144_Computer Science - Artificial Intelligence_2017_5_10.json
144_Computer Science - Artificial Intelligence_2017_5_11.json
144_Computer Science - Artificial Intelligence_2017_5_12.json
144_Computer Science - Artificial Intelligence_2017_5_13.json
144_Computer Science - Artificial Intelligence_2017_5_14.json
144_Computer Science - Artificial Intelligence_2017_5_15.json
144_Computer Science - Artificial Intelligence_2017_5_16.json
144_Computer Science - Artificial Intelligence_2017_5_17.json
144_Computer Science - Artificial Intelligence_2017_5_18.json
144_Computer Science - Artificial Intelligence_2017_5_19.json
144_Computer Science - Artificial Intelligence_2017_5_2.json
144_Computer Science - Artificial Intelligence_2017_5_20.json
144_Computer Science - Artificial Intelligence_2017_5_21.json
144_Computer Science - Artificial Intelligence_2017_5_22.json
144_Compute

It seems like they are not individual articles, so we have to look into one of them.
Let's just take the last one.

In [42]:
json_file = os.path.join(path, filename)
with open(json_file) as f:
    data = json.load(f)

JSON files are loaded as dictionaries:

In [43]:
data.keys()

dict_keys(['authors', 'contributors', 'datePublished', 'description', 'doi', 'downloadUrl', 'fullText', 'fulltextIdentifier', 'id', 'identifiers', 'oai', 'relations', 'repositories', 'subjects', 'title', 'topics', 'types', 'year'])

If we look into one of those keys, it seems like there is many articles, each one with their respective value for the keys above.

In [53]:
data['title'].keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47'])

So we can check the content of the `Title` key, for all articles:

In [61]:
for key in data['title']:
    print(data['title'][key])
    print('-'*50)

A new cosine series antialiasing function and its application to
  aliasing-free glottal source models for speech and singing synthesis
--------------------------------------------------
Investigating the role of musical genre in human perception of music
  stretching resistance
--------------------------------------------------
Enabling Early Audio Event Detection with Neural Networks
--------------------------------------------------
Multi-Speaker Localization Using Convolutional Neural Network Trained
  with Noise
--------------------------------------------------
Acoustic Reflector Localization: Novel Image Source Reversion and Direct
  Localization Methods
--------------------------------------------------
A modulation property of time-frequency derivatives of filtered phase
  and its application to aperiodicity and fo estimation
--------------------------------------------------
Weakly Supervised PLDA Training
--------------------------------------------------
CNN Architectures f

Or the content of all keys, for the first article, which is what we are looking for:

In [60]:
for key in data:
    print(key, data[key]['0'])
    print('-'*50)

authors ['Kawahara, Hideki', 'Sakakibara, Ken-Ichi', 'Banno, Hideki', 'Morise, Masanori', 'Toda, Tomoki', 'Irino, Toshio']
--------------------------------------------------
contributors []
--------------------------------------------------
datePublished 2017-06-08
--------------------------------------------------
description We formulated and implemented a procedure to generate aliasing-free
excitation source signals. It uses a new antialiasing filter in the continuous
time domain followed by an IIR digital filter for response equalization. We
introduced a cosine-series-based general design procedure for the new
antialiasing function. We applied this new procedure to implement the
antialiased Fujisaki-Ljungqvist model. We also applied it to revise our
previous implementation of the antialiased Fant-Liljencrants model. A
combination of these signals and a lattice implementation of the time varying
vocal tract model provides a reliable and flexible basis to test fo extractors
and sourc

Awesome! So we have to iterate for all the downloaded JSON files, for all keys, for all articles, so we can store it in a database. In this case we will use MongoDB.

## Extraction and Storage - MongoDB

We use MongoDB because it is great for prototyping and quick schema changes, but other storage systems can be probably used. Ultimately, all the data is then indexed in Elasticsearch, and that's where we actually query in production.

You can go [here](https://docs.mongodb.com/manual/installation/) to install MongoDB.

Then, using a MongoDB client, create a database, for example _pub_ and a collection, like _publications_.

In [92]:
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError
import string

In [70]:
# Default values
mongoDB_IP = '127.0.0.1'
mongoDB_Port = 27017
mongoDB_db = 'pub'

In [71]:
def connect_to_mongo():
    """
    Returns a db connection to the mongo instance
    :return:
    """
    try:
        client = MongoClient(mongoDB_IP, mongoDB_Port)
        db = client[mongoDB_db]
        db.downloads.find_one({'_id': 'test'})
        return db
    except ServerSelectionTimeoutError as e:
        raise Exception("Local MongoDB instance at "+mongoDB_IP+":"+mongoDB_Port+" could not be accessed.") from e

We can test the connection to the database with the following command:

In [72]:
db = connect_to_mongo()

Then we define a function that will take each of the articles from the JSON files downloaded from CORE, and store the information into our collection.

The data obtained from CORE does not separate sections in the text, it is not so important except for the references, since we may use them later on, therefore we also define a function that will manage this.

In [147]:
def arxiv_json_to_mongo(article):
    """
    Creates a new entry in mongodb from the input article
    :return:
    """
    
    mongo_input = {}
    translator = str.maketrans('', '', string.punctuation)
    article_data = article

    mongo_input['title'] = article_data['title']
    mongo_input['authors'] = article_data['authors']
    mongo_input['journal'] = 'arxiv'
    mongo_input['year'] = article_data['year']
    mongo_input['type'] = article_data['subjects']
    mongo_input['content.abstract'] = article_data['description']
    mongo_input['content.keywords'] = article_data['topics']
    mongo_input['content.fulltext'] = article_data['fullText']
    mongo_input['content.references'] = article_data['references']

    mongo_mongo_input = db.publications.update_one(
        {'_id': 'arxiv_' + article_data['id']},
        {'$set': mongo_input},
        upsert=True
    )    
    print('Processed', article_data['id'], 'from JSON')
    
def manage_text_and_refs(article):
    article['fullText'] = article['fullText'].replace('\n', ' ').replace('\r', '')
    if len(article['fullText'].split('References', 1)) == 2:
        article['references'] = article['fullText'].split('References', 1)[1].split('[')
        article['fullText'] = article['fullText'].split('References', 1)[0]
    elif len(article['fullText'].split('REFERENCES', 1)) == 2:
        article['references'] = article['fullText'].split('REFERENCES', 1)[1].split('[')
        article['fullText'] = article['fullText'].split('References', 1)[0]
    else:
        article['fullText'] = article['fullText']
        article['references'] = ''
    return article

Now we have to iterate through all the downloaded JSON files, and all the articles in each file, to process them and introduce them into our database.

In [149]:
for dirpath, dirnames, filenames in os.walk(path):
    # Iterate files...
    for filename in filenames:
        if filename.endswith('.json'):
            json_file = os.path.join(path, filename)
            with open(json_file) as f:
                data = json.load(f)            
                    # Iterate articles in file...
                for article_number in data['title'].keys():
                    article = {}
                    for key in data.keys():
                        article[key] = data[key][str(article_number)]
                        
                    # Process text and references for each article
                    manage_text_and_refs(article)
                    # Store to database
                    arxiv_json_to_mongo(article)

Processed 25036267 from JSON
Processed 2113087 from JSON
Processed 25025591 from JSON
Processed 25015102 from JSON
Processed 24956790 from JSON
Processed 25038409 from JSON
Processed 2246166 from JSON
Processed 73386897 from JSON
Processed 83863968 from JSON
Processed 83867371 from JSON
Processed 94047681 from JSON
Processed 146472595 from JSON
Processed 94063315 from JSON
Processed 29526388 from JSON
Processed 29536914 from JSON
Processed 84090473 from JSON
Processed 84093559 from JSON
Processed 84093915 from JSON
Processed 84093852 from JSON
Processed 84094420 from JSON
Processed 84326602 from JSON
Processed 84328172 from JSON
Processed 84330324 from JSON
Processed 84329889 from JSON
Processed 84326664 from JSON
Processed 84331310 from JSON
Processed 86416235 from JSON
Processed 86419366 from JSON
Processed 86420205 from JSON
Processed 84330603 from JSON
Processed 84330526 from JSON
Processed 84331497 from JSON
Processed 73423992 from JSON
Processed 73417539 from JSON
Processed 73408

Processed 83866609 from JSON
Processed 93946032 from JSON
Processed 93954042 from JSON
Processed 93952440 from JSON
Processed 93874081 from JSON
Processed 93943848 from JSON
Processed 93944339 from JSON
Processed 93944694 from JSON
Processed 93945015 from JSON
Processed 93942643 from JSON
Processed 146474068 from JSON
Processed 146474486 from JSON
Processed 42744287 from JSON
Processed 42715942 from JSON
Processed 42730154 from JSON
Processed 94072332 from JSON
Processed 94071067 from JSON
Processed 94058844 from JSON
Processed 93907363 from JSON
Processed 146475319 from JSON
Processed 84331874 from JSON
Processed 86415709 from JSON
Processed 86415925 from JSON
Processed 86418777 from JSON
Processed 84332442 from JSON
Processed 84330749 from JSON
Processed 84331069 from JSON
Processed 84330624 from JSON
Processed 84330701 from JSON
Processed 84331502 from JSON
Processed 84331595 from JSON
Processed 86415305 from JSON
Processed 86414737 from JSON
Processed 86417914 from JSON
Processed 7

Processed 93944988 from JSON
Processed 93947576 from JSON
Processed 93948719 from JSON
Processed 93951313 from JSON
Processed 93953333 from JSON
Processed 93953566 from JSON
Processed 93936930 from JSON
Processed 93936512 from JSON
Processed 93940234 from JSON
Processed 93869650 from JSON
Processed 93907518 from JSON
Processed 93942191 from JSON
Processed 93945152 from JSON
Processed 94032693 from JSON
Processed 146471726 from JSON
Processed 94024498 from JSON
Processed 129246622 from JSON
Processed 129264059 from JSON
Processed 129225652 from JSON
Processed 141534671 from JSON
Processed 141535333 from JSON
Processed 141534796 from JSON
Processed 93883528 from JSON
Processed 83845686 from JSON
Processed 83848072 from JSON
Processed 83849835 from JSON
Processed 83852895 from JSON
Processed 83859106 from JSON
Processed 83864340 from JSON
Processed 83856208 from JSON
Processed 83858660 from JSON
Processed 73383291 from JSON
Processed 83869383 from JSON
Processed 83841305 from JSON
Process

Processed 86421012 from JSON
Processed 86419000 from JSON
Processed 86417477 from JSON
Processed 86417617 from JSON
Processed 86419961 from JSON
Processed 29518418 from JSON
Processed 83831911 from JSON
Processed 83834080 from JSON
Processed 83833917 from JSON
Processed 83834840 from JSON
Processed 83832976 from JSON
Processed 83833715 from JSON
Processed 146475980 from JSON
Processed 146473775 from JSON
Processed 94062508 from JSON
Processed 94050938 from JSON
Processed 146474949 from JSON
Processed 94070006 from JSON
Processed 94075467 from JSON
Processed 94074604 from JSON
Processed 129357551 from JSON
Processed 93940465 from JSON
Processed 83836132 from JSON
Processed 83835487 from JSON
Processed 83835069 from JSON
Processed 42669201 from JSON
Processed 42706142 from JSON
Processed 42719258 from JSON
Processed 74203909 from JSON
Processed 74251340 from JSON
Processed 42743024 from JSON
Processed 74251078 from JSON
Processed 73960926 from JSON
Processed 73960988 from JSON
Processed 

Processed 42714190 from JSON
Processed 29509195 from JSON
Processed 84094933 from JSON
Processed 73442277 from JSON
Processed 86414492 from JSON
Processed 42641564 from JSON
Processed 42688561 from JSON
Processed 42655868 from JSON
Processed 42742913 from JSON
Processed 74202836 from JSON
Processed 74203978 from JSON
Processed 74203947 from JSON
Processed 42743606 from JSON
Processed 129239884 from JSON
Processed 129243931 from JSON
Processed 73993085 from JSON
Processed 42730879 from JSON
Processed 42739232 from JSON
Processed 42737724 from JSON
Processed 129270085 from JSON
Processed 129269247 from JSON
Processed 129274513 from JSON
Processed 94031902 from JSON
Processed 141529941 from JSON
Processed 141530082 from JSON
Processed 74250873 from JSON
Processed 74251720 from JSON
Processed 78509678 from JSON
Processed 141533043 from JSON
Processed 141532723 from JSON
Processed 78512366 from JSON
Processed 146472859 from JSON
Processed 94046603 from JSON
Processed 93908494 from JSON
Proc

...

Processed 94025848 from JSON
Processed 74251795 from JSON
Processed 74251252 from JSON
Processed 146473522 from JSON
Processed 146472257 from JSON
Processed 94056134 from JSON
Processed 129354224 from JSON
Processed 129357494 from JSON
Processed 129363237 from JSON
Processed 141532659 from JSON
Processed 93907882 from JSON
Processed 93915955 from JSON
Processed 93947042 from JSON
Processed 93946830 from JSON
Processed 93943824 from JSON
Processed 129361680 from JSON
Processed 129224567 from JSON
Processed 129355676 from JSON
Processed 93886529 from JSON
Processed 129203131 from JSON
Processed 129238032 from JSON
Processed 78510782 from JSON
Processed 73387570 from JSON
Processed 73357496 from JSON
Processed 83846492 from JSON
Processed 83866522 from JSON
Processed 83869172 from JSON
Processed 73378699 from JSON
Processed 73355678 from JSON
Processed 83841358 from JSON
Processed 83833425 from JSON
Processed 83836277 from JSON
Processed 73353766 from JSON
Processed 83845615 from JSON
Pro

Processed 83860964 from JSON
Processed 83859582 from JSON
Processed 83861563 from JSON
Processed 83862270 from JSON
Processed 83855807 from JSON
Processed 83851160 from JSON
Processed 83849848 from JSON
Processed 83863862 from JSON
Processed 73360346 from JSON
Processed 83849088 from JSON
Processed 83847981 from JSON
Processed 93877417 from JSON
Processed 83866499 from JSON
Processed 83832849 from JSON
Processed 83838241 from JSON
Processed 83839167 from JSON
Processed 83853275 from JSON
Processed 73393242 from JSON
Processed 73361148 from JSON
Processed 83847160 from JSON
Processed 73420526 from JSON
Processed 73407389 from JSON
Processed 73408623 from JSON
Processed 84092329 from JSON
Processed 84092855 from JSON
Processed 73420728 from JSON
Processed 84094969 from JSON
Processed 73953774 from JSON
Processed 84330096 from JSON
Processed 84327283 from JSON
Processed 42680346 from JSON
Processed 73992924 from JSON
Processed 73987595 from JSON
Processed 84333196 from JSON
Processed 8432

Processed 83833450 from JSON
Processed 94034127 from JSON
Processed 94075171 from JSON
Processed 129353838 from JSON
Processed 93940465 from JSON
Processed 83840614 from JSON
Processed 74251340 from JSON
Processed 42698180 from JSON
Processed 73958868 from JSON
Processed 84330224 from JSON
Processed 84331955 from JSON
Processed 84331272 from JSON
Processed 73990846 from JSON
Processed 73993061 from JSON
Processed 42728177 from JSON
Processed 42736156 from JSON
Processed 93908285 from JSON
Processed 93907530 from JSON
Processed 93912161 from JSON
Processed 93867227 from JSON
Processed 78508000 from JSON
Processed 93946665 from JSON
Processed 83860612 from JSON
Processed 83852880 from JSON
Processed 83853061 from JSON
Processed 83852789 from JSON
Processed 83855500 from JSON
Processed 83847817 from JSON
Processed 83850365 from JSON
Processed 83855719 from JSON
Processed 83842711 from JSON
Processed 129233569 from JSON
Processed 83865840 from JSON
Processed 83870426 from JSON
Processed 83

Processed 94047681 from JSON
Processed 84094468 from JSON
Processed 84093340 from JSON
Processed 86421626 from JSON
Processed 84326431 from JSON
Processed 86415559 from JSON
Processed 84329252 from JSON
Processed 73414888 from JSON
Processed 73405938 from JSON
Processed 78507488 from JSON
Processed 141531952 from JSON
Processed 141532753 from JSON
Processed 93891203 from JSON
Processed 93936890 from JSON
Processed 93887776 from JSON
Processed 93942772 from JSON
Processed 42638983 from JSON
Processed 83839161 from JSON
Processed 73384954 from JSON
Processed 73394009 from JSON
Processed 83857668 from JSON
Processed 83865306 from JSON
Processed 83846248 from JSON
Processed 83850977 from JSON
Processed 73377839 from JSON
Processed 83864132 from JSON
Processed 94078807 from JSON
Processed 129358344 from JSON
Processed 129361018 from JSON
Processed 129243468 from JSON
Processed 73961238 from JSON
Processed 83838221 from JSON
Processed 83847342 from JSON
Processed 86419132 from JSON
Processed

Processed 86420799 from JSON
Processed 129240605 from JSON
Processed 141531716 from JSON
Processed 93900853 from JSON
Processed 129356830 from JSON
Processed 93949731 from JSON
Processed 93948310 from JSON
Processed 83835194 from JSON
Processed 146474357 from JSON
Processed 73374050 from JSON
Processed 93937091 from JSON
Processed 146471442 from JSON
Processed 94041308 from JSON
Processed 94049681 from JSON
Processed 146473431 from JSON
Processed 83867553 from JSON
Processed 73384129 from JSON
Processed 83848680 from JSON
Processed 78509187 from JSON
Processed 141538825 from JSON
Processed 73441981 from JSON
Processed 84093274 from JSON
Processed 93955721 from JSON
Processed 83831680 from JSON
Processed 93888084 from JSON
Processed 129235802 from JSON
Processed 86422136 from JSON
Processed 84331601 from JSON
Processed 86415268 from JSON
Processed 146471597 from JSON
Processed 141534242 from JSON
Processed 141538453 from JSON
Processed 141532549 from JSON
Processed 83844191 from JSON
Pr

Done! We have a bunch of files in our database! (8848 in the demo)

## Indexing - Elasticsearch 

We use Elasticsearch for the quick and very flexible (elastic?) queries across the full text of articles, so we have to index all the content from the database for this.

Once again, we have to [install](https://www.elastic.co/guide/en/elasticsearch/reference/current/_installation.html) it and run the service...yay!

Once it's running we can connect and check status:

In [182]:
import pymongo
import elasticsearch
import requests
import nltk
from elasticsearch import helpers

In [157]:
client = pymongo.MongoClient('localhost:' + str(mongoDB_Port))
publications_collection = client.pub.publications
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)

In [155]:
es.cluster.health()

{'active_primary_shards': 0,
 'active_shards': 0,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

In [156]:
def extract_metadata(documents):
    list_of_docs = []
    for i, r in enumerate(documents):
        extracted = {
                "_id": "",
                "title": "",
                "publication": "",
                "year": "",
                "content": "",
                "abstract": "",
                "authors": [],
                "references": []}
        extracted['_id'] = r['_id']
        extracted['title'] = r['title']
        extracted['publication'] = r['journal']
        extracted['year'] = r['year']
        extracted['content'] = r['content']['fulltext']
        extracted['abstract'] = r['content']['abstract']
        extracted['authors'] = r['authors']
        extracted['references'] = r['content']['references']
        list_of_docs.append(extracted)
    return list_of_docs

### Index the full text of all articles in the database

In [162]:
filter_publications = ['arxiv'] # Here we could also put PubMed or other sources

extracted_publications = []
for publication in filter_publications:
    query ={'$and': [{'journal': publication}, {'content.fulltext': {'$exists': True}}]}                   
    results = publications_collection.find(query)
    extracted_publications.append(extract_metadata(results))

In [166]:
for publication in extracted_publications:
    actions = []
    for article in publication:
        authors = []
        if len(article['authors']) > 0:
            if type(article['authors'][0]) == list:
                try:
                    for name in article['authors']:
                        authors.append(', '.join([name[1], name[0]]))
                    authors = list(set(authors))
                except:
                    pass
            else:
                authors = article['authors']
        actions.append({
            "_index": "ir", 
            "_type": "publications",  
            "_id": article['_id'],
            "_source": {
                "title": article["title"],
                "journal": article['publication'],
                "year": str(article['year']),
                "content": article["content"],
                "abstract": article["abstract"],
                "authors": authors,
                "references": article["references"]
            }
        })
    if len(actions) == 0:
        continue
    res = helpers.bulk(es, actions)
    print('Done with', res, 'articles added to index')

Done with (8848, []) articles added to index


We can look for anything in the text, like this:

In [180]:
res = es.search(index = "ir", body = {"query": {"match": {"title" : "netflix"}}}, size = 10)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print(hit['_id'], hit['_source']['abstract'])
    print('-'*50)

Got 1 Hits:
arxiv_84094279 Re-Evaluating the Netflix Prize - Human Uncertainty and its Impact on
  Reliability
arxiv_84094279 In this paper, we examine the statistical soundness of comparative
assessments within the field of recommender systems in terms of reliability and
human uncertainty. From a controlled experiment, we get the insight that users
provide different ratings on same items when repeatedly asked. This volatility
of user ratings justifies the assumption of using probability densities instead
of single rating scores. As a consequence, the well-known accuracy metrics
(e.g. MAE, MSE, RMSE) yield a density themselves that emerges from convolution
of all rating densities. When two different systems produce different RMSE
distributions with significant intersection, then there exists a probability of
error for each possible ranking. As an application, we examine possible ranking
errors of the Netflix Prize. We are able to show that all top rankings are more
or less subject to h

### Sentence Indexing 
For the sentence expansion step of our process, we need to have sentences indexed. This is because we need to find similar sentences as well as their surrounding sentences, for context. 

In addition, we create a file with all the text, of all our articles, and we will use it later for the training of embedding models.

In [199]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [201]:
f.close()

In [ ]:
for publication in extracted_publications:
    for article in publication:
        actions = []
        cleaned = []
        dataset_sent = []
        other_sent = []

        lines = (sent_detector.tokenize(article['content'].strip()))
        
        # This will be used for the training of word2vec and doc2vec
        # You need to create the data folder beforehand
        with open('data/full_text_corpus.txt', 'a', encoding='utf-8') as f:
            for line in lines:
                f.write(line)
        f.close()
        
        if len(lines) < 3:
            continue

        for i in range(len(lines)):
            words = nltk.word_tokenize(lines[i])
            word_lengths = [len(x) for x in words]
            average_word_length = sum(word_lengths) / len(word_lengths)
            if average_word_length < 4:
                continue

            two_sentences = ''
            try:
                two_sentences = lines[i] + ' ' + lines[i - 1]
            except:
                two_sentences = lines[i] + ' ' + lines[i + 1]

            dataset_sent.append(two_sentences)

        for num, added_lines in enumerate(dataset_sent):
            actions.append({
                "_index": "twosent",
                "_type": "twosentnorules",
                "_id": article['_id'] + str(num),
                "_source": {
                    "title": article['title'],
                    "content.chapter.sentpositive": added_lines,
                    "paper_id": article['_id']
                }})

        if len(actions) == 0:
            continue
        res = helpers.bulk(es, actions)
    print('Done')

In [193]:
res = es.search(index = "twosent", body = {"query": {"match": {"content.chapter.sentpositive" : "netflix"}}}, size = 10)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print('-'*50)

Got 174 Hits:
arxiv_14153268986 Learning to Rank based on Analogical Reasoning
--------------------------------------------------
arxiv_146472211389 One-sided Differential Privacy
--------------------------------------------------
arxiv_129348860158 Differentially Private Query Learning: from Data Publishing to Model
  Publishing
--------------------------------------------------
arxiv_14153268985 Learning to Rank based on Analogical Reasoning
--------------------------------------------------
arxiv_129348860154 Differentially Private Query Learning: from Data Publishing to Model
  Publishing
--------------------------------------------------
arxiv_129348860153 Differentially Private Query Learning: from Data Publishing to Model
  Publishing
--------------------------------------------------
arxiv_93942601272 Interacting Attention-gated Recurrent Networks for Recommendation
--------------------------------------------------
arxiv_129348860157 Differentially Private Query Learning: from

### Doc2vec Indexing

Once again, we index the sentences from the full text file created before, which will be used for the Sentence Expansion.

In [197]:
file = open('data/full_text_corpus.txt', 'r', encoding='utf-8')
text = file.read()
file.close()
sentences = nltk.tokenize.sent_tokenize(text)
print('Sentences ready')
count = 0
docLabels = []
actions = []

for i, sent in enumerate(sentences):
    try:
        neighbors = sentences[i + 1]
        neighbor_count = count + 1
    except:
        neighbors = sentences[i - 1]
        neighbor_count = count - 1

    docLabels.append(count)
    actions.append({
        "_index": "devtwosentnew",
        "_type": "devtwosentnorulesnew",
        "_id": count,
        "_source": {
            "content.chapter.sentpositive": sent,
            "content.chapter.sentnegtive": neighbors,
            "neighborcount": neighbor_count
        }})
    count = count + 1

print(len(sentences))
print(len(docLabels))
res = helpers.bulk(es, actions)
print(res)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 1144: invalid start byte